# Texas Tow Trucks

We're going to scrape some [tow trucks in Texas](https://www.tdlr.texas.gov/tools_search/).

## Import your imports

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

import pandas as pd

## Search for the TLDR Number `006179570C`, and scrape the information on that company

Using [license information system](https://www.tdlr.texas.gov/tools_search/), find information about the tow truck number above, displaying the

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

If you can't figure a 'nice' way to locate something, your two last options might be:

- **Find a "parent" element, then dig inside**
- **Find all of a type of element** (like we did with `td` before) and get the `[0]`, `[1]`, `[2]`, etc
- **XPath** (inspect an element, Copy > Copy XPath)

These kinds of techniques tend to break when you're on other result pages, but... maybe not! You won't know until you try.

> - *TIP: When you use xpath, you CANNOT use double quotes or Python will get confused. Use single quotes.*
> - *TIP: You can clean your data up if you want to, or leave it dirty to clean later*
> - *TIP: The address part can be tough, but you have a few options. You can use a combination of `.split` and list slicing to clean it now, or clean it later in the dataframe with regular expressions. Or other options, too, probably*

In [2]:
driver = webdriver.Chrome()

In [3]:
url = 'https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C'
driver.get(url)

In [4]:
# business = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[2]/td[1]')
# business.text
# --> xpath worked, but I was afraid it wouldn't once I start scraping for more tow trucks, so
# commented it out and tried this:

# First found all tables on search result page:
tables = driver.find_elements_by_tag_name('table')

# I kept printing text from different tables by changing [index] until hit right one, [3]
company_info_table = tables[3]

# finding td's so can zero in on table cells with text I need:
company_info_rows = company_info_table.find_elements_by_tag_name('tr')

IndexError: list index out of range

In [ ]:
# Getting name:
name = company_info_rows[1].find_elements_by_tag_name('td')[0]
print(name.text.split(":")[1])

In [ ]:
# Getting owner/operator:
owner_operator = company_info_rows[2].find_elements_by_tag_name('td')[0]
print(owner_operator.text.split(":")[1])

In [ ]:
# Getting phone number:
phone = company_info_rows[3].find_elements_by_tag_name('td')[0]
print(phone.text)

In [ ]:
# Getting license status (Active, Expired, Etc):

# Moving to new table with more info I need:
cert_table = tables[4]
#print(cert_table.text)

# Finding all rows in that table:
cert_table_rows = cert_table.find_elements_by_tag_name('tr')

license_status = cert_table_rows[0].find_elements_by_tag_name('td')[1].find_element_by_tag_name('font').text
print(license_status)

In [ ]:
# Getting physical address:
# physical_address = cert_table_rows[1].find_elements_by_tag_name('td')[1].find_elements_by_tag_name('strong')[3].text.split("\n")
# physical_address

# ("").join(driver.find...RESTOFCODE) --> takes the result of all my code, which is a list, as
# a result of my .split, and combines them into a string
# the (" ") at beg tells python to put a space between the two list indices i'm joining (can put whatever you want there, a . or a $)

physical_address = " ".join(driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[2]/td[2]').text.split("\n")[-2:])
physical_address

# Adapt this to work inside of a single cell

Double-check that it works. You want it to print out all of the details.

In [11]:
url = 'https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber='
driver.get(url + "006179570C")

# variables
tables = driver.find_elements_by_tag_name('table')
company_info_table = tables[3]
company_info_rows = company_info_table.find_elements_by_tag_name('tr')
cert_table = tables[4]
cert_table_rows = cert_table.find_elements_by_tag_name('tr')

name = company_info_rows[1].find_elements_by_tag_name('td')[0].text.split(":   ")[1]
owner_operator = company_info_rows[2].find_elements_by_tag_name('td')[0].text.split(":   ")[1]
phone = company_info_rows[3].find_elements_by_tag_name('td')[0].text.split(":   ")[1]
license_status = cert_table_rows[0].find_elements_by_tag_name('td')[1].find_element_by_tag_name('font').text
physical_address = " ".join(driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[2]/td[2]').text.split("\n")[-2:])

print(name)
print(owner_operator)
print(phone)
print(license_status)
print(physical_address)

B.D. SMITH TOWING
BRANDT SMITH / OWNER
8173330706
Expired
13619 BRETT JACKSON RD. FORT WORTH, TX. 76179


# Using .apply to find data about SEVERAL tow truck companies

The file `trucks-subset.csv` has information about the trucks, we'll use it to find the pages to scrape.

### Open up `trucks-subset.csv` and save it into a dataframe

In [23]:
df = pd.read_csv("trucks-subset.csv")
df.columns = df.columns.str.replace(' ', '_')
df.head()

,TDLR_Number
0,006507931C
1,006179570C
2,006502097C


## Go through each row of the dataset, displaying the URL you will need to scrape for the information on that row

You don't have to actually use the search form for each of these - look at the URL you're on, it has the number in it!

For example, one URL might look like `https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006495492C`.

- *TIP: Use .apply and a function*
- *TIP: Unlike the Yelp example, you'll need to build this URL from pieces*
- *TIP: You probably don't want to `print` unless you're going to fix it for the next question 
- *TIP: pandas won't showing you the entire url! Run `pd.set_option('display.max_colwidth', -1)` to display aaaalll of the text in a cell*

In [14]:
pd.set_option('display.max_colwidth', -1)

In [61]:
url = 'https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber='

def get_truck_url(row):
    truck_url = url + row
    
    return pd.Series({
        'url': truck_url
    })

In [60]:
# PRACTICE/TEST TO MAKE SURE WAS DOING THE ABOVE FUNCTION RIGHT
# for row in df.TDLR_Number:
#     truck_url = url + row
#     print(truck_url)

'https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C'

In [62]:
df['url'] = df.TDLR_Number.apply(get_truck_url)
df.head()

,TDLR_Number,url
0,006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C


### Save this URL into a new column of your dataframe, called `url`

- *TIP: Use a function and `.apply`*
- *TIP: Be sure to use `return`*

In [65]:
driver.get(df.url[2])

## Go through each row of the dataset, printing out information about each tow truck company.

Now will be **scraping** inside of your function.

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

Just print it out for now.

- *TIP: use .apply*
- *TIP: You'll be using the code you wrote before, but converted into a function*
- *TIP: Remember how the TDLR Number is in the URL? You don't need to do the form submission if you don't want!*
- *TIP: Make sure you adjust any variables so you don't scrape the same page again and again*

In [71]:
# I COULD NOT FIGURE OUT TO GET IT TO ONLY DISPLAY/RETURN THE RESULTS; COULD ONLY GET IT TO 
# DO WHAT NEXT QUES ASKS, TO JOIN IT WITH OLD DF

def get_company_info(row):
    driver.get(row['url'])
    
    # variables
    tables = driver.find_elements_by_tag_name('table')
    company_info_table = tables[3]
    company_info_rows = company_info_table.find_elements_by_tag_name('tr')
    cert_table = tables[4]
    cert_table_rows = cert_table.find_elements_by_tag_name('tr')

    name = company_info_rows[1].find_elements_by_tag_name('td')[0].text.split(":   ")[1]
    owner_operator = company_info_rows[2].find_elements_by_tag_name('td')[0].text.split(":   ")[1]
    phone = company_info_rows[3].find_elements_by_tag_name('td')[0].text.split(":   ")[1]
    license_status = cert_table_rows[0].find_elements_by_tag_name('td')[1].find_element_by_tag_name('font').text
    physical_address = " ".join(driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[2]/td[2]').text.split("\n")[-2:])
    
    return pd.Series({
        'name': name,
        'owner_operator': owner_operator,
        'phone': phone,
        'license_status': license_status,
        'physical_address': physical_address
    })
    
#     return name
#     return owner_operator
#     return phone
#     return license_status
#     return physical_address


In [73]:
# df.url.apply(get_company_info)

driver = webdriver.Chrome()

# Take every row and send it to get_yelp_info, and combine with old data
new_df = df.apply(get_company_info, axis=1).join(df)
new_df.head()

,name,owner_operator,phone,license_status,physical_address,TDLR_Number,url
0,AUGUSTUS E SMITH,AUGUSTUS EUGENE SMITH / OWNER,9032276464,Active,"103 N MAIN ST BONHAM, TX. 75418",006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,B.D. SMITH TOWING,BRANDT SMITH / OWNER,8173330706,Expired,"13619 BRETT JACKSON RD. FORT WORTH, TX. 76179",006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,BARRY MICHAEL SMITH,BARRY MICHAEL SMITH / OWNER,8066544404,Active,"4501 W CEMETERY RD CANYON, TX. 79015",006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C


## Scrape the following information for each row of the dataset, and save it into new columns in your dataframe.

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

It's basically what we did before, but using the function a little differently.

- *TIP: Same as above, but you'll be returning a `pd.Series` and the `.apply` line is going to be a lot longer*
- *TIP: Save it to a new dataframe!*
- *TIP: Make sure you change your `df` variable names correctly if you're cutting and pasting - there are a few so it can get tricky*

### Save your dataframe as a CSV named `tow-trucks-extended.csv`

In [74]:
new_df.to_csv("tow-trucks-extended.csv", index=False)

### Re-open your dataframe to confirm you didn't save any extra weird columns

In [75]:
df = pd.read_csv("tow-trucks-extended.csv")
df.head()

,name,owner_operator,phone,license_status,physical_address,TDLR_Number,url
0,AUGUSTUS E SMITH,AUGUSTUS EUGENE SMITH / OWNER,9032276464,Active,"103 N MAIN ST BONHAM, TX. 75418",006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,B.D. SMITH TOWING,BRANDT SMITH / OWNER,8173330706,Expired,"13619 BRETT JACKSON RD. FORT WORTH, TX. 76179",006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,BARRY MICHAEL SMITH,BARRY MICHAEL SMITH / OWNER,8066544404,Active,"4501 W CEMETERY RD CANYON, TX. 79015",006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C


## Process the entire `tow-trucks.csv` file

We just did it on a short subset so far. Now try it on all of the tow trucks. **Save as the same filename as before**

In [81]:
df = pd.read_csv("tow-trucks.csv")
df.columns = df.columns.str.replace(' ', '_')
df.head()

,TDLR_Number
0,006507931C
1,006179570C
2,006502097C
3,006494912C
4,0649468VSF


In [82]:
url = 'https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber='

def get_truck_url(row):
    truck_url = url + row
    
    return pd.Series({
        'url': truck_url
    })

In [83]:
df['url'] = df.TDLR_Number.apply(get_truck_url)
df.head()

,TDLR_Number,url
0,006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C
3,006494912C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006494912C
4,0649468VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0649468VSF


In [84]:
def get_company_info(row):
    driver.get(row['url'])
    
    # variables
    tables = driver.find_elements_by_tag_name('table')
    company_info_table = tables[3]
    company_info_rows = company_info_table.find_elements_by_tag_name('tr')
    cert_table = tables[4]
    cert_table_rows = cert_table.find_elements_by_tag_name('tr')

    name = company_info_rows[1].find_elements_by_tag_name('td')[0].text.split(":   ")[1]
    owner_operator = company_info_rows[2].find_elements_by_tag_name('td')[0].text.split(":   ")[1]
    phone = company_info_rows[3].find_elements_by_tag_name('td')[0].text.split(":   ")[1]
    license_status = cert_table_rows[0].find_elements_by_tag_name('td')[1].find_element_by_tag_name('font').text
    physical_address = " ".join(driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[2]/td[2]').text.split("\n")[-2:])
    
    return pd.Series({
        'name': name,
        'owner_operator': owner_operator,
        'phone': phone,
        'license_status': license_status,
        'physical_address': physical_address
    })

In [87]:
driver = webdriver.Chrome()

new_df = df.apply(get_company_info, axis=1).join(df)

In [88]:
new_df

,name,owner_operator,phone,license_status,physical_address,TDLR_Number,url
0,AUGUSTUS E SMITH,AUGUSTUS EUGENE SMITH / OWNER,9032276464,Active,"103 N MAIN ST BONHAM, TX. 75418",006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,B.D. SMITH TOWING,BRANDT SMITH / OWNER,8173330706,Expired,"13619 BRETT JACKSON RD. FORT WORTH, TX. 76179",006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,BARRY MICHAEL SMITH,BARRY MICHAEL SMITH / OWNER,8066544404,Active,"4501 W CEMETERY RD CANYON, TX. 79015",006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C
3,HEATH SMITH,HEATH A SMITH / OWNER,940-552-0687,Expired,"1529 WILBARGER ST VERNON, TX. 76384",006494912C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006494912C
4,HEATH SMITH,HEATH A SMITH / OWNER,9405520687,Expired,"1529 WILBARGER ST VERNON, TX. 76384",0649468VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0649468VSF
5,HYSMITH AUTOMOTIVE,WILLIAM THOMAS HYSMITH / PRESIDENT,ASHLEY ERIN HYSMITH / TREASURER,Active,"1210 US 380 BYPASS GRAHAM, TX. 76450",006448786C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006448786C
6,HYSMITH AUTOMOTIVE & TRUCK REPAIR INC,HYSMITH ERIN ASHLEY / TREASURER,WILLIAM THOMAS HYSMITH / PRESIDENT,Suspended,"927 LOVING HWY GRAHAM, TX. 76450",0648444VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0648444VSF
7,HYSMITH AUTOMOTIVE & TRUCK REPAIR INC,WILLIAM THOMAS HYSMITH / PRESIDENT,ASHLEY ERIN HYSMITH / TREASURER,Active,"1210 380 BYPASS GRAHAM, TX. 76450",0651667VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0651667VSF
8,JEFF & WENDY SMITH,JEFF SMITH / PARTNER,WENDY SMITH / PARTNER,Suspended,"10842 FM 2138 N JACKSONVILLE, TX. 75766",006017767C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006017767C
9,JEFF SMITH,JEFFREY JOHN SMITH / OWNER,8324354670,Active,"4338 HARVEY RD CROSBY, TX. 77532",006495492C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006495492C


In [89]:
new_df.to_csv("tow-trucks-extended.csv", index=False)

In [90]:
df = pd.read_csv("tow-trucks-extended.csv")
df

,name,owner_operator,phone,license_status,physical_address,TDLR_Number,url
0,AUGUSTUS E SMITH,AUGUSTUS EUGENE SMITH / OWNER,9032276464,Active,"103 N MAIN ST BONHAM, TX. 75418",006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,B.D. SMITH TOWING,BRANDT SMITH / OWNER,8173330706,Expired,"13619 BRETT JACKSON RD. FORT WORTH, TX. 76179",006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,BARRY MICHAEL SMITH,BARRY MICHAEL SMITH / OWNER,8066544404,Active,"4501 W CEMETERY RD CANYON, TX. 79015",006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C
3,HEATH SMITH,HEATH A SMITH / OWNER,940-552-0687,Expired,"1529 WILBARGER ST VERNON, TX. 76384",006494912C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006494912C
4,HEATH SMITH,HEATH A SMITH / OWNER,9405520687,Expired,"1529 WILBARGER ST VERNON, TX. 76384",0649468VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0649468VSF
5,HYSMITH AUTOMOTIVE,WILLIAM THOMAS HYSMITH / PRESIDENT,ASHLEY ERIN HYSMITH / TREASURER,Active,"1210 US 380 BYPASS GRAHAM, TX. 76450",006448786C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006448786C
6,HYSMITH AUTOMOTIVE & TRUCK REPAIR INC,HYSMITH ERIN ASHLEY / TREASURER,WILLIAM THOMAS HYSMITH / PRESIDENT,Suspended,"927 LOVING HWY GRAHAM, TX. 76450",0648444VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0648444VSF
7,HYSMITH AUTOMOTIVE & TRUCK REPAIR INC,WILLIAM THOMAS HYSMITH / PRESIDENT,ASHLEY ERIN HYSMITH / TREASURER,Active,"1210 380 BYPASS GRAHAM, TX. 76450",0651667VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0651667VSF
8,JEFF & WENDY SMITH,JEFF SMITH / PARTNER,WENDY SMITH / PARTNER,Suspended,"10842 FM 2138 N JACKSONVILLE, TX. 75766",006017767C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006017767C
9,JEFF SMITH,JEFFREY JOHN SMITH / OWNER,8324354670,Active,"4338 HARVEY RD CROSBY, TX. 77532",006495492C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006495492C
